# Exloratory Data Analysis (EDA)

## Import Necessary Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import Dataset

In [2]:
FILE_PATH = 'C:/Users/gaelm/OneDrive - DePaul University/Gael/Trabajo/Fraud Detection System/data/raw/'
FILE_NAME = 'PS_20174392719_1491204439457_log.csv'
FULL_PATH = os.path.join(FILE_PATH, FILE_NAME)

In [10]:
# Read Data
df = pd.read_csv(FULL_PATH)
df.head(10) # --> Make sure the data was correctly read

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


## Basic Initial Exploration

In [4]:
# Getting basic information
print("--- Data's Basic Info ---")
df.info()

--- Data's Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


The initial dataset consists of 6362620 obervations, 10 predictors (see types below) and a target feature (isFraud).
- 3 are qualitative ('object' type)
- 8 are quantitative (3 'int64' and 5 'float64')

In [5]:
# Initial Missig Values
print("--- Missing Values ---")
df.isnull().sum()

--- Missing Values ---


step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

There are no missing values in the dataset.

In [6]:
# Target Feature Distribution
print("--- Target Feature Distribution ---")
target_distribution = df['isFraud'].value_counts()
classes = []
total_observations = len(df)
for i in target_distribution:
    option = (i/total_observations)*100
    classes.append(option)

print(f"{target_distribution[0]} legitimate transctions ({classes[0]:.2f}%).")
print(f"{target_distribution[1]} fraudulent transactions ({classes[1]:.2f}%).")

--- Target Feature Distribution ---
6354407 legitimate transctions (99.87%).
8213 fraudulent transactions (0.13%).


There is a great class imbalance in the dataset, with less than 1% of the observations being fraudulent. 
While this was expected, class balancing techniques (like SMOTE) must be applied to work with proper data.

## Relationships with Target Feature

In [7]:
# Relationship between type and isFraud
print("--- Fraudulent Transactions by Transaction Type ---")
transaction_types = df['type'].unique()
fraud_transactions_by_type = df[df['isFraud'] == 1]['type'].value_counts()

print(f"There are {len(transaction_types)}: {transaction_types[0]}, {transaction_types[1]}, {transaction_types[2]}, {transaction_types[3]}, {transaction_types[4]}") 
for transaction_type, count in fraud_transactions_by_type.items():
    print(f"{count} fraudulent transactions were made by {transaction_type}")

--- Fraudulent Transactions by Transaction Type ---
There are 5: PAYMENT, TRANSFER, CASH_OUT, DEBIT, CASH_IN
4116 fraudulent transactions were made by CASH_OUT
4097 fraudulent transactions were made by TRANSFER


There are 5 types of transactions: 'payment', 'transfer', 'cash-out', 'cash-in', and 'debit'. However, fraud is only present in 'transfer' and 'cash-out' transactions.  

In [38]:
# Relationship between amount and isFraud
difference = 0

print("--- Fraudulent Amount Statistics ---")
avg_fraudulent_amount = df[df['isFraud'] == 1]['amount'].mean()
avg_legitimate_amount = df[df['isFraud'] == 0]['amount'].mean()
print(f"Average Fradulent Transaction: ${avg_fraudulent_amount:,.2f}\nAverage Legitimate Transaction: ${avg_legitimate_amount:,.2f}")
if avg_fraudulent_amount > avg_legitimate_amount:
    difference = avg_fraudulent_amount - avg_legitimate_amount
    print(f"The average fraudulent transaction is ${difference:,.2f} bigger than the average legitimate transaction.")
else:
    difference = avg_legitimate_amount - avg_fraudulent_amount
    print(f"The average legitimate transaction is ${difference:,.2f} bigger than the average fraudulent transaction.")

min_fraudulent_amount = df[df['isFraud'] == 1]['amount'].min()
min_legitimate_amount = df[df['isFraud'] == 0]['amount'].min()
print(f"\nLowest Fradulent Transaction: ${min_fraudulent_amount:,.2f}\nLowest Legitimate Transaction: ${min_legitimate_amount:,.2f}")
if min_fraudulent_amount > min_legitimate_amount:
    difference = min_fraudulent_amount - min_legitimate_amount
    print(f"The smallest fraudulent transaction is ${difference:,.2f} bigger than the smallest legitimate transaction.")
else:
    difference = min_legitimate_amount - min_fraudulent_amount
    print(f"The smallest legitimate transaction is ${difference:,.2f} bigger than the smallest fraudulent transaction.")

max_fraudulent_amount = df[df['isFraud'] == 1]['amount'].max()
max_legitimate_amount = df[df['isFraud'] == 0]['amount'].max()
print(f"\nHighest Fradulent Transaction: ${max_fraudulent_amount:,.2f}\nHighest Legitimate Transaction: ${max_legitimate_amount:,.2f}")
if max_fraudulent_amount > max_legitimate_amount:
    difference = max_fraudulent_amount - avg_legitimate_amount
    print(f"The biggest fraudulent transaction is ${difference:,.2f} bigger than the biggest legitimate transaction.")
else:
    difference = max_legitimate_amount - max_fraudulent_amount
    print(f"The biggest legitimate transaction is ${difference:,.2f} bigger than the biggest fraudulent transaction.")

--- Fraudulent Amount Statistics ---
Average Fradulent Transaction: $1,467,967.30
Average Legitimate Transaction: $178,197.04
The average fraudulent transaction is $1,289,770.26 bigger than the average legitimate transaction.

Lowest Fradulent Transaction: $0.00
Lowest Legitimate Transaction: $0.01
The smallest legitimate transaction is $0.01 bigger than the smallest fraudulent transaction.

Highest Fradulent Transaction: $10,000,000.00
Highest Legitimate Transaction: $92,445,516.64
The biggest legitimate transaction is $82,445,516.64 bigger than the biggest fraudulent transaction.


- The average fraudulent transaction is $1,289,770.26 bigger than the average legitimate transaction.

- The smallest legitimate transaction is $0.01 bigger than the smallest fraudulent transaction.

- The biggest legitimate transaction is $82,445,516.64 bigger than the biggest fraudulent transaction.

In [60]:
# Relationship between step and isFraud
print("--- Fraudulent Transactions by Hour ---")
hour_name_map = {0: '12:00 - 12:59 a.m.', 1: '1:00 - 1:59 a.m.', 2: '2:00 - 2:59 a.m.', 3: '3:00 - 3:59 a.m.', 
                 4: '4:00 - 4:59 a.m.', 5: '5:00 - 5:59 a.m.', 6: '6:00 - 6:59 a.m.', 7: '7:00 - 7:59 a.m.', 
                 8: '8:00 - 8:59 a.m.', 9: '9:00 - 9:59 a.m.', 10: '10:00 - 10:59 a.m.', 11: '11:00 - 11:59 a.m.', 
                 12: '12:00 - 12:59 p.m.', 13: '1:00 - 1:59 p.m.', 14: '2:00 - 2:59 p.m.', 15: '3:00 - 3:59 p.m.', 
                 16: '4:00 - 4:59 p.m.', 17: '5:00 - 5:59 p.m.', 18: '6:00 - 6:59 p.m.', 19: '7:00 - 7:59 p.m.', 
                 20: '8:00 - 8:59 p.m.', 21: '9:00 - 9:59 p.m.', 22: '10:00 - 10:59 p.m.', 23: '11:00 - 11:59 p.m.',}
df['hour_of_day'] = (df['step'] - 1) % 24
fraud_hours = df[df['isFraud'] == 1]['hour_of_day'].value_counts().sort_index()
for hour_num, count in fraud_hours.items():
    hour_name = hour_name_map.get(hour_num)
    print(f"{hour_name}:\t {count}")
top_3_fraud_hours_nums = fraud_hours.nlargest(3).index.tolist()
top_3_fraud_hours_names = [hour_name_map.get(hour_num) for hour_num in top_3_fraud_hours_nums]
top_3_fraud_hours_names.sort()
print(f"The 3 hours with the most fraudulent transactions are {', '.join(top_3_fraud_hours_names)}.")

print("\n--- Fraudulent Transactions by Week Day ---")
day_name_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
df['day_of_month'] = (df['step'] - 1) // 24 + 1
df['day_of_week'] = (df['day_of_month'] - 1) % 7 + 1
fraud_days_week = df[df['isFraud'] == 1]['day_of_week'].value_counts().sort_index()
for day_num, count in fraud_days_week.items():
    day_name = day_name_map.get(day_num)
    print(f"{day_name}: \t{count}")

top_3_fraud_week_days = fraud_days_week.nlargest(3).index.tolist()
top_3_fraud_week_days_names = [day_name_map.get(day_num) for day_num in top_3_fraud_week_days]
top_3_fraud_week_days_names.sort()
print(f"The 3 days of the week with the most fraudulent transactions are {', '.join(top_3_fraud_week_days_names)}.")

print("\n--- Fraudulent Transactions by Month Day ---")
fraud_days_month = df[df['isFraud'] == 1]['day_of_month'].value_counts().sort_index()
for day_num, count in fraud_days_month.items():
    print(f"Day {day_num}:\t{count}")
top_3_fraud_month_days_nums = fraud_days_month.nlargest(3).index.tolist()
top_3_fraud_month_days_nums.sort()
top_3_fraud_month_days_str = [str(day) for day in top_3_fraud_month_days_nums]
print(f"The 3 days of the month with the most fraudulent transactions are: {', '.join(top_3_fraud_month_days_str)}")

--- Fraudulent Transactions by Hour ---
12:00 - 12:59 a.m.:	 358
1:00 - 1:59 a.m.:	 372
2:00 - 2:59 a.m.:	 326
3:00 - 3:59 a.m.:	 274
4:00 - 4:59 a.m.:	 366
5:00 - 5:59 a.m.:	 358
6:00 - 6:59 a.m.:	 328
7:00 - 7:59 a.m.:	 368
8:00 - 8:59 a.m.:	 341
9:00 - 9:59 a.m.:	 375
10:00 - 10:59 a.m.:	 324
11:00 - 11:59 a.m.:	 339
12:00 - 12:59 p.m.:	 346
1:00 - 1:59 p.m.:	 353
2:00 - 2:59 p.m.:	 341
3:00 - 3:59 p.m.:	 345
4:00 - 4:59 p.m.:	 353
5:00 - 5:59 p.m.:	 343
6:00 - 6:59 p.m.:	 342
7:00 - 7:59 p.m.:	 340
8:00 - 8:59 p.m.:	 347
9:00 - 9:59 p.m.:	 351
10:00 - 10:59 p.m.:	 323
11:00 - 11:59 p.m.:	 300
The 3 hours with the most fraudulent transactions are 1:00 - 1:59 a.m., 7:00 - 7:59 a.m., 9:00 - 9:59 a.m..

--- Fraudulent Transactions by Week Day ---
Monday: 	1315
Tuesday: 	1300
Wednesday: 	1464
Thursday: 	1032
Friday: 	1078
Saturday: 	986
Sunday: 	1038
The 3 days of the week with the most fraudulent transactions are Monday, Tuesday, Wednesday.

--- Fraudulent Transactions by Month Day ---

- The 3 hours with the most fraudulent transactions are 1:00 - 1:59 a.m., 7:00 - 7:59 a.m., 9:00 - 9:59 a.m..
- The 3 days of the week with the most fraudulent transactions are Monday, Tuesday, Wednesday.
- The 3 days of the month with the most fraudulent transactions are: 2, 3, 17

Note: The order shown is alphabetical, it does not reflect the actual order (from highest to lowest frauds).

In [65]:
# Relationship between balances (origin and destination) and isFraud
df_fraud = df[df['isFraud'] == 1].copy()

print("--- Balance Inconsistencies ---")
# Check for balance inconsistencies (Originator)
df_fraud['expected_newbalanceOrg'] = df_fraud['oldbalanceOrg'] - df_fraud['amount']
df_fraud['balance_diff_orig'] = df_fraud['newbalanceOrig'] - df_fraud['expected_newbalanceOrg']
inconsistent_orig_balances = df_fraud[(df_fraud['type'].isin(['TRANSFER', 'CASH_OUT'])) & 
    (df_fraud['balance_diff_orig'].abs() > 0.01)]
print(f"Number of fraudulent transactions with inconsistent balances (Origin): {len(inconsistent_orig_balances)}")

# Check for balance inconsistencies (Destination)
df_fraud['expected_newbalanceDest'] = df_fraud['oldbalanceDest'] + df_fraud['amount']
df_fraud['balance_diff_dest'] = df_fraud['newbalanceDest'] - df_fraud['expected_newbalanceDest']
inconsistent_dest_balances = df_fraud[(df_fraud['type'].isin(['TRANSFER'])) & 
    (~df_fraud['nameDest'].str.startswith('M')) & 
    (df_fraud['balance_diff_dest'].abs() > 0.01)]
print(f"Number of fraudulent transactions with inconsistent balances (Destination): {len(inconsistent_dest_balances)}")

if len(inconsistent_orig_balances) > len(inconsistent_dest_balances):
    print(f"The Origin shows more balance inconsistencies than the Destination.")
else:
    print(f"The Destination shows more balance inconsistencies than the Origin.")

# Specific Fraud Pattern: Origin Account Emptied
print("\n--- Origin's Account Emptied ---")
fraud_orig_emptied = df_fraud[
    (df_fraud['type'].isin(['TRANSFER', 'CASH_OUT'])) & 
    (df_fraud['oldbalanceOrg'] > 0) & 
    (df_fraud['newbalanceOrig'] == 0)]
print(f"Number of TRANSFER/CASH_OUT where origin account was emptied: {len(fraud_orig_emptied)}")

# Specific Fraud Pattern: Mule Account Behavior (Destination starts at 0, ends at 0 after transaction)
print("\n--- Mule Account (Destination starts at 0, ends at 0 after transaction) ---")
fraud_mule_account = df_fraud[
    (df_fraud['type'].isin(['TRANSFER', 'CASH_OUT'])) & 
    (df_fraud['oldbalanceDest'] == 0) & 
    (df_fraud['newbalanceDest'] == 0) &
    (df_fraud['amount'] > 0)]
print(f"Number of TRANSFER/CASH_OUT with mule account behavior: {len(fraud_mule_account)}")

# Specific Fraud Pattern: High Amount with Zero Balances (Strongest signal in this dataset)
print("\n--- High Amount with Zero Balances ---")
fraud_zero_balances_high_amount = df_fraud[
    (df_fraud['type'].isin(['TRANSFER', 'CASH_OUT'])) &
    (df_fraud['oldbalanceOrg'] == 0) & 
    (df_fraud['newbalanceOrig'] == 0) & 
    (df_fraud['oldbalanceDest'] == 0) & 
    (df_fraud['newbalanceDest'] == 0) &
    (df_fraud['amount'] > 0)]
print(f"Number of TRANSFER/CASH_OUT with all balances zero and high amount: {len(fraud_zero_balances_high_amount)}")

--- Balance Inconsistencies ---
Number of fraudulent transactions with inconsistent balances (Origin): 45
Number of fraudulent transactions with inconsistent balances (Destination): 4095
The Destination shows more balance inconsistencies than the Origin.

--- Origin's Account Emptied ---
Number of TRANSFER/CASH_OUT where origin account was emptied: 8012

--- Mule Account (Destination starts at 0, ends at 0 after transaction) ---
Number of TRANSFER/CASH_OUT with mule account behavior (dest 0->0): 4070

--- High Amount with Zero Balances ---
Number of TRANSFER/CASH_OUT with all balances zero and high amount: 0


The Destination shows more balance inconsistencies than the Origin. Keeping in mind that fraudulent transactions in this dataset come only from transfers and cash-outs, we can conclude that:
- A good portion of the total fraudulent activities come from the 'transfers' on the Destination's account.
    - From this transfers, the Origin's account was emptied 8012 times.
    - The Destination's account started and ended with $0 balance 4070 times.

In [34]:
# Relationship between isFlaggedFraud and isFraud
print("--- Illegal Attemps to Transfer more than $200,000 ---")
flagged_transactions = (df['isFlaggedFraud'] == 1).sum()
df_fraud = df[df['isFraud'] == 1].copy()
correctly_flagged = (df_fraud['isFlaggedFraud'] == 1).sum()
if flagged_transactions == correctly_flagged:
    print("All transactions flagged as fraudulent were correclty flagged.")
else:
    print(f"Out of {flagged_transactions} transactions flagged as fraudulent (transfer over $200,000), {correctly_flagged} were correclty flagged.")

--- Illegal Attemps to Transfer more than $200,000 ---
All transactions flagged as fraudulent were correclty flagged.


Based on the observations on this dataset, we can conclude that transfers of $200,000 or more are 100% fraudulent.